### Utils

In [1]:
import json
import mlflow
import os

def pretty_print(d):
    print(f'=======\n{json.dumps(d, sort_keys=True, indent=4)}\n======')

home_dir = os.path.expanduser('~')
mlflow_dir = os.path.join(home_dir, 'git', 'rattaca', 'app', 'flask', 'mlruns')
mlflow.set_tracking_uri(f'file://{mlflow_dir}')
# print(mlflow.get_tracking_uri())

create_expiriments = False
if create_expiriments:
    iris_exp = mlflow.create_experiment("iris")
    housing_exp = mlflow.create_experiment("experiment1")

### Classifier Example

In [2]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


print(mlflow_dir)
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target
df.to_csv('iris_data.csv')

X_train, X_test, y_train, y_test = train_test_split(df[iris.feature_names], iris.target, test_size=0.2, random_state=42)


rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

eval_data = X_test
eval_data['label'] = y_test

run_name = 'iris_experiment_run'
with mlflow.start_run(run_name=run_name, description='iris sample', experiment_id=iris_exp):

    mlflow.sklearn.log_model(rf, 'iris_model')
    model_uri = mlflow.get_artifact_uri('iris_model')
    
    mlflow.log_params(rf.get_params())
    
    result = mlflow.evaluate(
        model_uri,
        eval_data,
        targets="label",
        model_type="classifier",
        evaluators=["default"],
    )
pretty_print(result.metrics)


/Users/leonardoclo/git/rattaca/app/flask/mlruns


NameError: name 'iris_exp' is not defined

In [ ]:
print(mlflow.get_tracking_uri())

### Regresion example (With custom metrics)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np
import mlflow
from mlflow.models import make_metric
import os
import matplotlib.pyplot as plt

# loading the California housing dataset
cali_housing = fetch_california_housing(as_frame=True)
df = pd.DataFrame(data=cali_housing.data, columns=cali_housing.feature_names)
df['target'] = cali_housing.target
df.to_csv('cali_housing.csv')

# split the dataset into train and test partitions
X_train, X_test, y_train, y_test = train_test_split(
    cali_housing.data, cali_housing.target, test_size=0.2, random_state=123
)

# train the model
lin_reg = LinearRegression().fit(X_train, y_train)

# creating the evaluation dataframe
eval_data = X_test.copy()
eval_data["target"] = y_test


def squared_diff_plus_one(eval_df, _builtin_metrics):
    """
    This example custom metric function creates a metric based on the ``prediction`` and
    ``target`` columns in ``eval_df`.
    """
    return np.sum(np.abs(eval_df["prediction"] - eval_df["target"] + 1) ** 2)


def sum_on_target_divided_by_two(_eval_df, builtin_metrics):
    """
    This example custom metric function creates a metric derived from existing metrics in
    ``builtin_metrics``.
    """
    return builtin_metrics["sum_on_target"] / 2


def prediction_target_scatter(eval_df, _builtin_metrics, artifacts_dir):
    """
    This example custom artifact generates and saves a scatter plot to ``artifacts_dir`` that
    visualizes the relationship between the predictions and targets for the given model to a
    file as an image artifact.
    """
    plt.scatter(eval_df["prediction"], eval_df["target"])
    plt.xlabel("Targets")
    plt.ylabel("Predictions")
    plt.title("Targets vs. Predictions")
    plot_path = os.path.join(artifacts_dir, "example_scatter_plot.png")
    plt.savefig(plot_path)
    return {"example_scatter_plot_artifact": plot_path}


with mlflow.start_run(run_name=run_name, experiment_id=housing_exp, description='hosing sample'):
    mlflow.sklearn.log_model(lin_reg, "model")
    model_uri = mlflow.get_artifact_uri("model")
    
    mlflow.log_params(lin_reg.get_params())
    
    result = mlflow.evaluate(
        model=model_uri,
        data=eval_data,
        targets="target",
        model_type="regressor",
        evaluators=["default"],
        custom_metrics=[
            make_metric(
                eval_fn=squared_diff_plus_one,
                greater_is_better=False,
            ),
            make_metric(
                eval_fn=sum_on_target_divided_by_two,
                greater_is_better=True,
            ),
        ],
        custom_artifacts=[prediction_target_scatter],
    )

pretty_print(result.metrics)